In [6]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [24]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/fw/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [25]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aniai, a robotic company and kitchen solutions provider based in NYC, recently secured $12 million in Pre-Series A funding. The funding round was led by InterVest, with contributions from SV Investment, Ignite Innovation from the UK, and existing investor Capstone Partners. This financial boost will enable Aniai to establish its first dedicated manufacturing facility, named Factory One, in South Korea. The primary objective behind this move is to ensure stable production and effective supply chain management for their flagship product, the Alpha Grill. The company's decision aligns with its ongoing expansion efforts in North America and globally.

Established in 2020 and helmed by CEO Gunpil Hwang, Aniai specializes in developing robotic solutions for cooking hamburgers. Their main product, the Alpha Grill, leverages robotics to enhance the efficiency of hamburger cooking. The grill features a double-sided design capable of cooking up to eight patties simultaneously or 200 patties per 

The text discusses Numbers Station, an intelligent data stack automation platform based in Menlo Park, CA, which has recently raised a total of $17.5 million in funding. The funding consists of a $12.5 million series A round led by Madrona, with participation from Norwest Venture Partners, Factory, and notable angel investors including Jeff Hammerbacher, the Co-founder of Cloudera. This series A funding follows a previous $5 million seed round. By utilizing foundation model technology developed at the Stanford AI lab, Numbers Station aims to empower data workers to efficiently generate insights by automating time-consuming data tasks, starting with data transformation work.

Source: Original text excerpt from a news release on factory automation.

---

Numbers Station, a Menlo Park, CA-based company, has secured $17.5 million in funding for its intelligent data stack automation platform. This funding includes a $12.5 million Series A round led by Madrona, with participation from Norwes

The text discusses Software Defined Automation, a company that has recently secured $10 million in seed funding. Insight Partners led the funding round, with additional contributions from Baukunst VC, Fly Ventures, and First Momentum. Jon Rosenbaum from Insight Partners and Axel Bichara from Baukunst will now join the company's Board of Directors as part of the transaction.

Software Defined Automation specializes in revolutionizing factory automation through its Industrial-Control-as-a-Service (ICaaS) offering. This approach involves cloud-based management of existing Programmable Logic Controllers (PLCs) for technical operations (TechOps), Git-enabled PLC code versioning and collaboration for development operations (DevOps), and virtualization of PLCs on edge servers for Virtual PLC. By integrating these technologies, the company aims to dismantle proprietary silos in control technology stacks and facilitate the adoption of API-based modern microservices architecture.

The company ha

In [26]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [27]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are Senior Vice President of a Venture Capital Company. When you evaluate a new start-ups/business idea, you strictly adhere to the following guiding principles:
        - Aim for large profits with small investment
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Ensure all initiatives align with business profitability.
        - Mention what trend/products are you planning to invest
        - Mention what would be a favourable return from such investment
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        """

summary_solutions = []

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```

    You want to generate a memo to explain why would you invest in such business. Kindly follow these guidelines
    - Be realistic and specific
    - Use certain indicators for your reasons
    - Elaborate your decision in detail
    - Do consider about challenges in business when making decision
    - Analyse the business and technology risk for your investment
    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion.choices[0].message.content)

**Memo**

**Subject:** Investment Analysis and Decision on Software Defined Automation

**To:** Executive Board

**From:** [Your Name], Senior Vice President, [Venture Capital Company Name]

**Date:** [Date]

**Introduction**

After a thorough analysis of the recent seed funding round of Software Defined Automation, a company that is pioneering in the field of factory automation through its Industrial-Control-as-a-Service (ICaaS) offering, I have outlined below the rationale behind considering an investment in this venture. This memo details the potential for large profits, alignment with our investment principles, and an evaluation of the associated risks.

**Investment Rationale**

1. **High Growth Potential with Modest Investment:** The factory automation market is on a significant growth trajectory, fueled by the global push towards manufacturing efficiency and digital transformation. Software Defined Automation's $10M seed funding is a modest investment considering the potential m

In [28]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

First, repeat the following points for the first solution as found in the following text: {summary_solutions}

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating.append(completion.choices[0].message.content)


### Solution Evaluation: Software Defined Automation

- **Name:** Software Defined Automation
- **Solved Problem:** Lack of seamless integration and communication between different factory automation systems and equipment.
- **Solution:** Offering Industrial-Control-as-a-Service (ICaaS) that leverages existing PLC systems for improved integration and communication in factory automation.
- **Impact:** Streamlines factory automation processes, reduces downtime, and enhances performance by enabling software-defined control and virtualization of PLCs.
- **Technology:** Cloud-based management, Git-enabled PLC code versioning, and virtualization of PLCs on edge servers.
- **Sources:** Memo to Executive Board from Senior Vice President, [Venture Capital Company Name].

#### Ratings:

- **Functionality and Performance:** 3  
The solution directly addresses the core issue of integration and communication in factory automation, promising efficiency and performance improvements.

- **Scalability:

In [29]:
user_prompt_3 = f""""

Please take all the solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest absolute rating out of 15 (please be very diligent in this task. If there is an overall rating of 13, it should for example be outputed instead of an idea with a score of 10 out of 15). Sort the three in descending rating order.

Repeat the followinng points for each of the top three:

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”
    - also include their rating from: {initial_rating} to double check they are in the top 3 highest rated


"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

Based on the provided information, there is only one solution detailed for evaluation, which inherently makes it the highest-rated solution with an overall rating of 14 out of 15. Here are the details for the "Software Defined Automation" solution:

- **Name:** Software Defined Automation
- **Solved Problem:** Lack of seamless integration and communication between different factory automation systems and equipment.
- **Solution:** Offering Industrial-Control-as-a-Service (ICaaS) that leverages existing PLC systems for improved integration and communication in factory automation.
- **Impact:** Streamlines factory automation processes, reduces downtime, and enhances performance by enabling software-defined control and virtualization of PLCs.
- **Technology:** Cloud-based management, Git-enabled PLC code versioning, and virtualization of PLCs on edge servers.
- **Sources:** Memo to Executive Board from Senior Vice President, [Venture Capital Company Name].
- **Overall Rating:** 14 out of 